In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import decomposition
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from xgboost import XGBClassifier
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.metrics import mean_squared_error
import numpy as np
from pandas import read_csv
from sklearn.metrics import accuracy_score

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

In [ ]:
df = pd.read_csv("data_symptomes_checker_data (1).csv")#, header=0, sep=",")
df



# Missing value 

In [ ]:
df.isna().sum()

In [ ]:
df =df[df["result"]!="other"]

# Encoding Categorical Data

In [ ]:

# with get dumies : 
removeList = [	"Age_60_and_above",	"Gender",	"Test_indication"]
for i in removeList:
    df = pd.concat([df,pd.get_dummies(df[i], prefix=i)],axis=1)
df = df.drop(columns=removeList)


In [ ]:
from sklearn.preprocessing import LabelEncoder
print(df["result"].unique())
le = LabelEncoder()
#for feat in ["corona_result",	"age_60_and_above",	"gender",	"test_indication"]:
df["result"] = le.fit_transform(df["result"].astype(str))





In [ ]:
df =df.drop(columns=["Unnamed: 0"])

In [ ]:
df

In [ ]:
df=df.drop_duplicates()

In [ ]:
infection_list =df["result"].values
data=df.drop(columns=["result"])
data

# Distribution des classes :

In [ ]:
print( "nbre de valeurs pour 0 ",df[df["result"]==0].count()[1]/df.count()[1]*100)
print( "nbre de valeurs pour 1 ", df[df["result"]==1].count()[1]/df.count()[1]*100)

In [ ]:
import seaborn as sns
sns.histplot( x=infection_list)

In [ ]:
#j'essaye avec drop duplicate et sans duplication : 
#des modeles simples 
data

# Models

 ## Logistic Regression 

In [ ]:
std_slc = StandardScaler()
pca = decomposition.PCA()
logistic_Reg = LogisticRegression(penalty='l2', max_iter=100)

#('std_slc', std_slc),
pipe = Pipeline(steps=[
                      ('logistic_Reg', logistic_Reg)])

param_grid = {"logistic_Reg__C": np.logspace(-5, 5, 7)}

clf = GridSearchCV(pipe, param_grid, cv=StratifiedKFold(4), scoring='accuracy', verbose=2)
clf.fit(data, infection_list)
print(" ########################results##############################")
print(clf.best_score_)
print(clf.best_estimator_)
print(clf.best_params_)



 ## Decision Tree Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, infection_list, test_size=0.20, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(min_samples_leaf=5, max_depth=3)



clf=clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## LGBM Classifier with grid search 

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt    
from lightgbm import LGBMClassifier


model = LGBMClassifier()
parameters = {'max_depth' : sp_randInt(3, 10)
                 }

randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 5, n_iter = 30, n_jobs=-1)
randm.fit(data, infection_list)

#Random_search
print("\n The best estimator :\n",randm.best_estimator_)
    
print("\n The best score :\n",randm.best_score_)
    
print("\n The best parameters :\n", randm.best_params_)
  

    

## Xgboost Classifier



In [ ]:
XGBCL = XGBClassifier()
classifier = XGBCL.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
preds = classifier.predict(X_test)
feature_important = XGBCL.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
data.plot(kind='barh', color='brown')

In [ ]:
print(classification_report(preds, y_test))
cm = confusion_matrix(preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print(acc)
print(cm)

##Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf =RandomForestClassifier(n_estimators=10, random_state=42)
classifier = XGBCL.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
preds = classifier.predict(X_test)

In [ ]:
print(classification_report(preds, y_test))
cm = confusion_matrix(preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print(acc)
print(cm)

## Catboost

In [ ]:
pip install catboost

In [ ]:

import numpy as np
import catboost as cb

# train default classifier    
clf = cb.CatBoostClassifier()


clf= clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
preds = clf.predict(X_test)

print('error:',1-np.mean(preds==np.ravel(y_test)))


In [ ]:
print(classification_report(preds, y_test))
cm = confusion_matrix(preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print("accuracy", acc)
print(cm)

## Stacking models 

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from lightgbm import LGBMClassifier
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('lgbm', LGBMClassifier()),
    ('catboost',  cb.CatBoostClassifier()),
    ('xgb', XGBClassifier()),
    ('dec', DecisionTreeClassifier(min_samples_leaf=5, max_depth=3))
    
]
clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)


clf= clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)

In [ ]:
!pip install scikitplot

In [ ]:
print(score)
preds = clf.predict(X_test)
pred_proba= clf.predict_proba(X_test)
#print("predict prob ", pred_proba)

print( "Classification report" , classification_report(preds, y_test))
cm = confusion_matrix(preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print(acc)
print(cm)

#affichage matrice de confiance
class_names= ["Negative", "Positive"]
import matplotlib.pyplot as plt

metrics.plot_confusion_matrix(clf , X_test,y_test ,  display_labels=class_names,
                                 cmap=plt.cm.Blues)

In [ ]:
lr_probs = clf.predict_proba(X_test)
lr_probs = lr_probs[:, 1]
ns_probs = [0 for _ in range(len(y_test))]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, lr_probs)
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Model: ROC AUC=%.3f' % (lr_auc))
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_probs)
import matplotlib.pyplot
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='stacking models ')
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()